In [ ]:

# -*- coding: utf-8 -*-
# 메인 슬롯 루프를 "로직 변경 없이" 독립 실행 가능하도록 외부 초기화만 추가한 노트북 셀입니다.
# 필요 시 상단 초기값만 수정하세요.

import numpy as np
import math, time

# =========================
# 외부 초기화 (필요 최소한)
# =========================
rng = np.random.default_rng(0)

L_Assoc_Req_in_Slot = 5

# 시뮬/시간
N_slot_total = 20000
Observation_Period = 100
Opt_Observation_TH = True
Next_Observation_Time_TH = Observation_Period

# STA/입퇴장
N_STA = 20
N_STAs_existing = N_STA  # Mode 9용
N_unassoc_In  = 2
N_unassoc_Out = 1
T_Period_InOut = 500
Next_T_unassoc_In  = 200
Next_T_unassoc_Out = 400

# UORA/옵션
Mode_UORA = 0           # 0,2,3,4,5,6,7,8,9 중 선택
Mode_OBO_BEB = True
Mode_OBO_XIXD = False
Opt_Random_N_RA_RU_0 = False

# RU/OCW 테이블
N_RU = 16
N_RA_RU_0    = 8     # assoc용 랜덤접근 RU 수
N_RA_RU_2045 = 4     # unassoc용 랜덤접근 RU 수
RU_AID = np.zeros(N_RU, dtype=int)   # 0 또는 2045 등 AID 맵 (필요시 수정)
# Tab_OCW: 첫 열(col0)=STA 개수 축, 나머지 열=RA-RU에 따른 OCW
# 여기서는 예시 테이블(선형 증가), 실제 테이블 대입 권장
rows = 64
cols = max(N_RA_RU_0, N_RA_RU_2045) + 2  # col0 + RU열들(1..)
Tab_OCW = np.zeros((rows, cols), dtype=float)
Tab_OCW[:,0] = np.linspace(0, 512, rows)  # STA 수 축
for c in range(1, cols):
    Tab_OCW[:,c] = 8.0 * (2**(c-1))       # 예시값: 8,16,32,...

def nearest_index_col1(tab, value):
    # col0(=STA 수)에 가장 가까운 행 인덱스
    col0 = tab[:,0]
    return int(np.argmin(np.abs(col0 - value)))

# CW/alpha
CWmin, CWmax = 8.0, 1024.0
Init_alpha, Min_alpha, Max_alpha = 1.0, 0.05, 4.0
Inc, Dec = 0.10, 0.10
alpha_g = 1.0  # Mode 2/3용 글로벌 알파

# PHY/시간/레이트(임시 합리값)
L_PHY = 56.0
L_BACK = 14.0
L_Trigger = 48.0
SIFS = 16.0
T_dft = 3.2
T_gi  = 0.8
U_slot = 9.0
N_ss = 1
# RU별 N_sd (서브캐리어 수 예시), MCS별 비트/심볼 & 부호율(예시)
N_sd = np.full(N_RU, 52.0)
N_bp = np.array([1,2,2,4,6,6,8,8,10,10,12,12], dtype=float)  # 0..11
Rcod = np.array([1/2,1/2,3/4,1/2,2/3,3/4,2/3,3/4,3/4,5/6,3/4,5/6], dtype=float)

# 공통/배열 초기화
Common_MCS = 5  # 0-base index
STA_Type = np.zeros(N_STA, dtype=int)   # 0=Unassoc, 1=Assoc, -1=Disable
L_MPDU = np.full(N_STA, 2000.0)
MCS = np.full(N_STA, Common_MCS, dtype=int)
STA_CWmin = np.full(N_STA, CWmin, dtype=float)
STA_CWmax = np.full(N_STA, CWmax, dtype=float)
STA_CW    = np.full(N_STA, CWmin, dtype=float)
STA_OBO   = np.floor(STA_CW * rng.random(N_STA))
STA_Assigned_RU = np.zeros(N_STA, dtype=int)
STA_Assoc_Delay = np.zeros((N_STA,3), dtype=float)
Sim_Rst = np.zeros((N_STA,4), dtype=float)   # [Access,Success,Collisions,sum(n_tf)]
n_tf = np.zeros(N_STA, dtype=float)

# alpha/Succ (Mode 4~8)
alpha = np.full(N_STA, Init_alpha, dtype=float)
Succ = np.zeros((N_STA,2), dtype=float)      # [success_streak, collision_streak]


# RU 통계
Sim_Rst_RU = np.zeros((N_RU,3), dtype=float) # [IDLE,SUCCESS,COLLISION]

# 관측/로그
log_slot_time = []
log_alpha = []
log_slot_time_SUC = [0.0]    # MATLAB 누적 모사 위해 0으로 시작
log_SUC = [0.0]
Observation_N_assoc = []

# Throughput 관측 플래그
Opt_Observation_TH = True
Next_Observation_Time_TH = Observation_Period

# =========================
# ====== 메인 슬롯 루프 =====
# ====== (로직 변경 없음) ====
# =========================
# 주신 코드 그대로 붙여넣기
# --------------------------------
# 메인 슬롯 루프
i = 1
start_t = time.time()
while i <= N_slot_total:
    # 주기적 입/퇴장
    if Next_T_unassoc_In <= i and N_unassoc_In > 0:
        STA_ID_Disable = np.where(STA_Type == -1)[0]
        if len(STA_ID_Disable) == N_unassoc_In:
            # 재활성화
            STA_Type[STA_ID_Disable] = 0
            Next_T_unassoc_In = i + T_Period_InOut
        else:
            # 새로 추가
            old_N = N_STA
            N_STA += int(N_unassoc_In)
            Next_T_unassoc_In = i + T_Period_InOut

            STA_Type = np.concatenate([STA_Type, np.zeros(int(N_unassoc_In), dtype=int)])
            L_MPDU = np.concatenate([L_MPDU, np.full(int(N_unassoc_In), 2000.0)])
            MCS = np.concatenate([MCS, np.full(int(N_unassoc_In), Common_MCS, dtype=int)])
            STA_CWmin = np.concatenate([STA_CWmin, np.full(int(N_unassoc_In), CWmin, dtype=float)])
            STA_CWmax = np.concatenate([STA_CWmax, np.full(int(N_unassoc_In), CWmax, dtype=float)])
            STA_CW    = np.concatenate([STA_CW,    np.full(int(N_unassoc_In), CWmin, dtype=float)])
            STA_OBO   = np.concatenate([STA_OBO,   np.floor(CWmin * rng.random(int(N_unassoc_In)))])
            STA_Assigned_RU = np.concatenate([STA_Assigned_RU, np.zeros(int(N_unassoc_In), dtype=int)])
            add_delay = np.zeros((int(N_unassoc_In),3), dtype=float)
            add_delay[:,0] = (Next_T_unassoc_In - T_Period_InOut)  # entering time (원 코드 의미 보존)
            STA_Assoc_Delay = np.vstack([STA_Assoc_Delay, add_delay])
            Sim_Rst = np.vstack([Sim_Rst, np.zeros((int(N_unassoc_In),4), dtype=float)])
            n_tf = np.concatenate([n_tf, np.zeros(int(N_unassoc_In), dtype=float)])

            if Mode_UORA in (4,5,6,7,8):
                alpha = np.concatenate([alpha, np.full(int(N_unassoc_In), Init_alpha, dtype=float)])
                if Mode_UORA in (5,6,7,8):
                    Succ = np.vstack([Succ, np.zeros((int(N_unassoc_In),2), dtype=float)])

    if Next_T_unassoc_Out <= i and N_unassoc_Out > 0:
        STA_ID_Assoc = np.where(STA_Type == 1)[0]
        Disable_STA_ID = []
        while len(Disable_STA_ID) < int(N_unassoc_Out) and len(STA_ID_Assoc) > 0:
            pick = rng.choice(STA_ID_Assoc)
            if pick not in Disable_STA_ID:
                Disable_STA_ID.append(pick)
                STA_Type[pick] = -1
        Next_T_unassoc_Out = i + T_Period_InOut

    # 현재 assoc/unassoc 집합
    STA_ID_Unassoc = np.where(STA_Type == 0)[0]
    STA_ID_Assoc   = np.where(STA_Type == 1)[0]

    # OBO 감소
    if Mode_UORA in (0,9):
        # ax UORA
        if Opt_Random_N_RA_RU_0:
            N_Random_RA_RU_0 = int(math.ceil(rng.random()*N_RA_RU_0))
            if STA_ID_Unassoc.size:
                STA_OBO[STA_ID_Unassoc] -= N_RA_RU_2045
            if STA_ID_Assoc.size:
                STA_OBO[STA_ID_Assoc]   -= N_Random_RA_RU_0
        else:
            if STA_ID_Unassoc.size:
                STA_OBO[STA_ID_Unassoc] -= N_RA_RU_2045
            if STA_ID_Assoc.size:
                STA_OBO[STA_ID_Assoc]   -= N_RA_RU_0

    elif Mode_UORA == 1:
        # not supported in 원본 코드(에러)
        raise RuntimeError('Mode_UORA == 1 not supported in provided script')

    elif Mode_UORA == 2:
        STA_OBO -= alpha_g * N_RU
        # RU 배정은 아래에서 공통 처리

    elif Mode_UORA == 3:
        STA_OBO -= alpha_g * N_RU

    elif Mode_UORA in (4,5,6,7,8):
        if STA_ID_Unassoc.size:
            STA_OBO[STA_ID_Unassoc] -= alpha[STA_ID_Unassoc] * N_RA_RU_2045
        if STA_ID_Assoc.size:
            if Opt_Random_N_RA_RU_0:
                # 주기 갱신은 루프 말미에서, 여기선 직전 값 사용
                # 그래도 원본에서 대부분 0이라 동일
                NR = N_RA_RU_0
            else:
                NR = N_RA_RU_0
            STA_OBO[STA_ID_Assoc] -= alpha[STA_ID_Assoc] * NR

        # 관찰 로깅
        if 'Opt_Observation_Alpha' in locals():
            if N_STA in [10,20,100]:
                log_slot_time.append(i)
                # MATLAB Target_STA=1 (1-base). 파이썬은 0-base로 0 인덱스가 첫 STA
                if N_STA >= 1:
                    log_alpha.append(alpha[0])

    # TF 발행 & RU 배정
    STA_ID_TX = np.where(STA_OBO <= 0)[0]
    STA_ID_TX_Unassoc = np.setdiff1d(STA_ID_TX, STA_ID_Assoc, assume_unique=False)
    STA_ID_TX_Assoc   = np.setdiff1d(STA_ID_TX, STA_ID_TX_Unassoc, assume_unique=False)

    if STA_ID_TX_Assoc.size:
        if Opt_Random_N_RA_RU_0:
            N_Random_RA_RU_0 = max(1, int(math.ceil(rng.random()*N_RA_RU_0)))
            STA_Assigned_RU[STA_ID_TX_Assoc] = rng.integers(1, N_Random_RA_RU_0+1, size=STA_ID_TX_Assoc.size)
        else:
            STA_Assigned_RU[STA_ID_TX_Assoc] = rng.integers(1, N_RA_RU_0+1, size=STA_ID_TX_Assoc.size)
    if STA_ID_TX_Unassoc.size:
        # +N_RA_RU_0 offset
        STA_Assigned_RU[STA_ID_TX_Unassoc] = rng.integers(N_RA_RU_0+1, N_RU+1, size=STA_ID_TX_Unassoc.size)

    if STA_ID_TX.size:
        N_TF = 1  # 누적 카운트가 필요한 경우 외부에서 관리 가능. 여기선 더미 증가 표기만.
        n_tf += 1

    # RU별 판정
    STA_ID_SUC = []
    STA_ID_COL = []
    for j in range(1, N_RU+1):
        ids = np.where(STA_Assigned_RU == j)[0]
        if ids.size == 0:
            Sim_Rst_RU[j-1, 1-1] += 1  # IDLE
            continue

        Sim_Rst[ids, 0] += 1  # Access

        if ids.size > 1:
            # Collision
            Sim_Rst[ids, 2] += 1
            Sim_Rst_RU[j-1, 3-1] += 1
            STA_ID_COL.extend(ids.tolist())

            # BEB / XIXD / Mode 9 처리
            if Mode_OBO_BEB:
                if Mode_UORA == 9:
                    # RU.AID(j) 별로 Tab_OCW에서 CW 재설정
                    if RU_AID[j-1] == 0 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        # assoc 집합 현재 크기 기준
                        assoc_cnt = int(np.sum(STA_Type == 1))
                        r = nearest_index_col1(Tab_OCW, assoc_cnt)
                        if Opt_Random_N_RA_RU_0:
                            col = (N_Random_RA_RU_0 + 1)  # +1 오프셋 (1열이 STA수)
                        else:
                            col = (N_RA_RU_0 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[ids] = newcw
                        # if 1 in ids:  # 필요 시 특정 STA 로깅
                        #     log_OCW_assoc = np.vstack([log_OCW_assoc, [i, newcw]])
                    elif RU_AID[j-1] == 2045 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        unassoc_cnt = int(np.sum(STA_Type == 0))
                        r = nearest_index_col1(Tab_OCW, unassoc_cnt)
                        col = (N_RA_RU_2045 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[ids] = newcw
                        # log_OCW_unassoc = np.vstack([log_OCW_unassoc, [i, newcw]])
                else:
                    # 기본 BEB: CW = (CW+1)*2 -1, 상한 CWmax
                    STA_CW[ids] = (STA_CW[ids] + 1.0) * 2.0 - 1.0
                    over = np.where(STA_CW[ids] >= STA_CWmax[ids])[0]
                    if over.size:
                        STA_CW[ids[over]] = STA_CWmax[ids[over]]
            elif Mode_OBO_XIXD:
                # 원본 옵션 자리 (비활성)
                pass

            # alpha 조정
            if Mode_UORA == 3:
                alpha_g = max(Min_alpha, alpha_g - Dec)
            elif Mode_UORA == 4:
                if STA_ID_Assoc.size:
                    alpha_ids = ids
                    alpha[alpha_ids] = np.maximum(Min_alpha, alpha[alpha_ids] - Dec)
            elif Mode_UORA == 5:
                Succ[ids,1] += 1  # collisions++
                Succ[ids,0] = 0   # successes reset
                alpha[ids] = np.maximum(Min_alpha, alpha[ids] - Dec * (2.0 ** (Succ[ids,1]-1)))
            elif Mode_UORA == 6:
                for kk in ids:
                    Succ[kk,1] += 1
                    if Succ[kk,0] > 0:
                        alpha[kk] = Init_alpha
                    else:
                        alpha[kk] = max(Min_alpha, alpha[kk] - Dec)
                    Succ[kk,0] = 0
            elif Mode_UORA in (7,8):
                Succ[ids,1] += 1
                Succ[ids,0]  = 0
                if Mode_UORA == 7:
                    alpha[ids] = np.maximum(Min_alpha, alpha[ids] - Dec)
                else:
                    alpha[ids] = np.maximum(Min_alpha, alpha[ids] - Dec * (2.0 ** (Succ[ids,1]-1)))

        else:
            # Success
            one = ids[0]
            if STA_Type[one] == 1:
                Sim_Rst[one, 1] += 1
            Sim_Rst_RU[j-1, 2-1] += 1
            Sim_Rst[one, 3] += n_tf[one]
            n_tf[one] = 0
            STA_ID_SUC.append(one)

            # 성공 시 CW 갱신
            if Mode_OBO_BEB:
                if Mode_UORA == 9:
                    if RU_AID[j-1] == 0 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        assoc_cnt = int(np.sum(STA_Type == 1))
                        r = nearest_index_col1(Tab_OCW, assoc_cnt)
                        if Opt_Random_N_RA_RU_0:
                            col = (N_Random_RA_RU_0 + 1)
                        else:
                            col = (N_RA_RU_0 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[one] = newcw
                        # if one == 1:
                        #     log_OCW_assoc = np.vstack([log_OCW_assoc, [i, newcw]])
                    elif RU_AID[j-1] == 2045 and (N_unassoc_In > 0 or N_unassoc_Out > 0):
                        unassoc_cnt = int(np.sum(STA_Type == 0))
                        r = nearest_index_col1(Tab_OCW, unassoc_cnt)
                        col = (N_RA_RU_2045 + 1)
                        col = min(col, Tab_OCW.shape[1]-1)
                        newcw = Tab_OCW[r, col]
                        STA_CW[one] = newcw
                        # log_OCW_unassoc = np.vstack([log_OCW_unassoc, [i, newcw]])
                else:
                    # 성공 시 CWmin으로
                    STA_CW[one] = STA_CWmin[one]
            elif Mode_OBO_XIXD:
                # 비활성
                pass

            # alpha 증가
            if Mode_UORA == 3:
                alpha_g = min(Max_alpha, alpha_g + Inc)
            elif Mode_UORA == 4:
                alpha[one] = min(Max_alpha, alpha[one] + Inc)
            elif Mode_UORA == 5:
                Succ[one,0] += 1
                Succ[one,1]  = 0
                alpha[one] = min(Max_alpha, alpha[one] + Inc * (2.0 ** (Succ[one,0]-1)))
            elif Mode_UORA == 6:
                Succ[one,0] += 1
                if Succ[one,1] > 0:
                    alpha[one] = Init_alpha + Inc
                else:
                    alpha[one] = min(Max_alpha, alpha[one] + Inc)
                Succ[one,1] = 0
            elif Mode_UORA in (7,8):
                Succ[one,0] += 1
                Succ[one,1]  = 0
                if Mode_UORA == 7:
                    alpha[one] = min(Max_alpha, alpha[one] + Inc * (2.0 ** (Succ[one,0]-1)))
                else:
                    alpha[one] = min(Max_alpha, alpha[one] + Inc)

    # TF가 있었으면 전송 시간 소모 및 Assoc 성공 처리
    if STA_ID_TX.size:
        # 데이터 레이트 계산 준비
        data_rates = np.zeros(STA_ID_TX.size, dtype=float)
        for idx_k, ksta in enumerate(STA_ID_TX):
            ru = STA_Assigned_RU[ksta]
            # 방어
            ru = max(1, min(N_RU, ru))
            mcs = MCS[ksta]
            # mcs는 0..11
            data_rates[idx_k] = (N_sd[ru-1] * N_bp[mcs] * Rcod[mcs] * N_ss) / (T_dft + T_gi)

        STA_ID_assoc_req = STA_ID_TX[STA_Type[STA_ID_TX] == 0]
        STA_ID_Data_TX   = STA_ID_TX[STA_Type[STA_ID_TX] == 1]

        # 시간 소모 계산 (원본 3 케이스 구조)
        if STA_ID_assoc_req.size and STA_ID_Data_TX.size:
            dur = int( np.max( np.ceil( (L_PHY + (8*L_MPDU[STA_ID_TX]/data_rates)) / U_slot ) ) ) \
                  + int( np.ceil( (2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot) )
            i += dur
        elif STA_ID_assoc_req.size:
            dur = int(L_Assoc_Req_in_Slot) \
                  + int( np.ceil( (2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot ) )
            i += dur
        elif STA_ID_Data_TX.size:
            dur = int( np.max( np.ceil( (L_PHY + (8*L_MPDU[STA_ID_TX]/data_rates)) / U_slot ) ) ) \
                  + int( np.ceil( (2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot) )
            i += dur
        else:
            raise RuntimeError('unexpected situation!')

        # Assoc 성공 처리 (ACK 받은 시점 기록)
        for ksta in STA_ID_assoc_req:
            if ksta in STA_ID_SUC:
                STA_Type[ksta] = 1
                ack_time = i - int(np.ceil((2*L_PHY + L_BACK + L_Trigger + 3*SIFS) / U_slot))
                STA_Assoc_Delay[ksta,1] = ack_time
                STA_Assoc_Delay[ksta,2] = STA_Assoc_Delay[ksta,1] - STA_Assoc_Delay[ksta,0]

        # reset for next TF
        STA_Assigned_RU[:] = 0

        Observation_N_assoc.append([i,
                                    float(np.sum(STA_Type == 0)),
                                    float(np.sum(STA_Type == 1))])

    else:
        # 아무도 TX 아님 → 슬롯 1 증가
        i += 1
        Observation_N_assoc.append([i,
                                    float(np.sum(STA_Type == 0)),
                                    float(np.sum(STA_Type == 1))])

    # Throughput 관측 주기 업데이트
    if Opt_Observation_TH and (Next_Observation_Time_TH is not None) and (Next_Observation_Time_TH <= i):
        # MATLAB 누적 덧셈 방식 모사 (증분 저장)
        log_slot_time_SUC.append(i - sum(log_slot_time_SUC))
        log_SUC.append( float(np.sum(Sim_Rst[:,1])) - sum(log_SUC) )
        Next_Observation_Time_TH += Observation_Period

    # 루프 말미: Opt_Random_N_RA_RU_0 갱신 및 Mode9 CW 재적용 (원본 타이밍)
    if Opt_Random_N_RA_RU_0:
        N_Random_RA_RU_0 = int(math.ceil(rng.random()*N_RA_RU_0))
        if Mode_UORA == 9 and N_STAs_existing > 0:
            # assoc 기존 STA들의 CW를 현재 랜덤 RA-RU에 맞춰 재설정
            r = nearest_index_col1(Tab_OCW, N_STAs_existing)
            col = (N_Random_RA_RU_0 + 1)
            col = min(col, Tab_OCW.shape[1]-1)
            newcw = Tab_OCW[r, col]
            STA_CW[:N_STAs_existing] = newcw

    # TF 있었던 STA들의 OBO 재시작
    if STA_ID_TX.size:
        # OBO = floor(CW * rand)
        STA_OBO[STA_ID_TX] = np.floor(STA_CW[STA_ID_TX] * rng.random(STA_ID_TX.size))

# 루프 끝

print("루프 종료. N_STA =", N_STA)
print("Sim_Rst 합계(Access,Success,Collision,sum(n_tf)):", np.sum(Sim_Rst, axis=0))
print("마지막 5개 Observation_N_assoc:", Observation_N_assoc[-5:])


루프 종료. N_STA = 58
Sim_Rst 합계(Access,Success,Collision,sum(n_tf)): [2947. 1003. 1854. 9665.]
마지막 5개 Observation_N_assoc: [[19809, 3.0, 54.0], [19871, 2.0, 55.0], [19933, 2.0, 55.0], [19995, 2.0, 54.0], [20057, 2.0, 54.0]]
